In [ ]:
# Verificar que el dataset se puede cargar correctamente
from ultralytics.data.utils import check_det_dataset
import yaml
import zipfile

# Cargar y verificar el archivo de configuración
with open("larc-2/data_fixed.yaml", "r") as f:
    data_config = yaml.safe_load(f)
    
print("Configuración del dataset:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Verificar que el dataset se puede cargar
try:
    dataset_info = check_det_dataset("larc-2/data_fixed.yaml")
    print("\n✅ Dataset verificado exitosamente!")
    print(f"Imágenes de entrenamiento: {len(dataset_info.get('train', []))}")
except Exception as e:
    print(f"\n❌ Error verificando dataset: {e}")

In [ ]:
import torch
import cv2
from roboflow import Roboflow
import dotenv
import os
print("GPU disponible:", torch.cuda.is_available())

!nvidia-smi

GPU disponible: True
Wed Aug 27 20:03:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.169                Driver Version: 570.169        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0             38W /   55W |    1201MiB /   8188MiB |     48%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [16]:
dotenv.load_dotenv()
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace(os.getenv("ROBOFLOW_WORKSPACE")).project(os.getenv("ROBOFLOW_PROJECT_ID"))
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [19]:
dotenv.load_dotenv()
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace("larc-hbgqy").project("larc-5pojj")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to larc-2 in yolov8:: 100%|██████████| 534/534 [00:00<00:00, 13196.86it/s]


In [1]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8n model
model = YOLO("yolov8n.pt")

# Train the model with the downloaded dataset
results = model.train(
    data="larc-2/data.yaml", 
    epochs=50,
    imgsz=640,
    batch=4,
    split=0.8,
    perspective=0.0005,
    scale=0.4,
    translate=0.04,
    degrees=5,
    shear=1,
    hsv_s=0.5,
    hsv_v=0.3,
    flipud=0.2,
    patience=15,
    save_period=20,
    name="larc-model",
    project="runs/detect",
)

Ultralytics 8.3.187 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7817MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=larc-2/data.yaml, degrees=5, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=larc-model8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0005, plots=True, pos

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
